In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
import wandb

import torch
from transformers import set_seed

from src.config import CONFIG
from src.dataset import SBICDataset
from src.train_utils import *

2023-10-23 11:16:44.672179: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-23 11:16:45.475171: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
tokenizer = make_tokinzer(CONFIG.hp)
model = make_model(CONFIG.hp, tokenizer)
checkpoints = "gpt2_10832_8_2"
# checkpoints = "distilgpt2_30720_16"
model.load_state_dict(torch.load(f"checkpoints/{checkpoints}.pt"))

List of all special token and its token_id:
 - ['<|endoftext|>', '<|sep|>', '<|pad|>', '<|offY|>', '<|offN|>', '<|sexY|>', '<|sexN|>', '<|intY|>', '<|intN|>', '<|grpY|>', '<|grpN|>', '<|ingrpN|>', '<|ingrpY|>']
 - [[50256], [50258], [50257], [50259], [50260], [50261], [50262], [50263], [50264], [50265], [50266], [50267], [50268]]
Model vocab resize: 50269
Model eos token: 50256
Model pad token: 50257
Model sep token: 50258


<All keys matched successfully>

In [24]:
from rouge import Rouge

rouge_metric = Rouge()

rouge_metric.get_scores("ciao", "ciao")

ValueError: Reference is empty.

In [6]:
n_samples = {
    "train": 1024,
    "validation": 1024
}

for split in ["train", "validation"]:
    data = get_data(split)[:n_samples[split]]
    dataset = SBICDataset(data, tokenizer, is_training=False)
    dataloader = make_dataloader(dataset, model, tokenizer, CONFIG.hp, split="validation")

    res = evaluate(model, tokenizer, dataloader, CONFIG.hp)

    annotation_type = ["Offensive", "Intent", "Sex", "Group", "In-Group"]
    print(f"Classification F1 on {split} set: avg={np.mean(res['clasification_f1_score']):.3f}")
    for type, score in zip(annotation_type, res['clasification_f1_score']):
        print(f" - {type}: {score:.3f}")
    print(f"Minority RougeL-f1 on {split} set: {res['minority_rouge_f1_score']:.3f}")
    print(f"Stereotype RougeL-f1 on {split} set: {res['stereotype_rouge_f1_score']:.3f}")

[50259 50259 50259 50259 50259 50259 50259 50259]
[50260 50260 50260 50259 50259 50259 50259 50259]
[0, 1, 2, 3, 4, 5, 6, 7]


RuntimeError: No active exception to reraise

Classification F1 on train set: avg=0.590
 - Offensive: 0.577
 - Intent: 0.508
 - Sex: 0.787
 - Group: 0.425
 - In-Group: 0.652
Minority RougeL-f1 on train set: 0.690
Stereotype RougeL-f1 on train set: 0.336
                                                 
Classification F1 on validation set: avg=0.578
 - Offensive: 0.579
 - Intent: 0.490
 - Sex: 0.790
 - Group: 0.453
 - In-Group: 0.575
Minority RougeL-f1 on validation set: 0.519
Stereotype RougeL-f1 on validation set: 0.243